# Solver Benchmark - Finding All Solutions

We analyze how fast various solvers are in listing all solutions for a simple AND and a simple OR formula.
Solvers might not natively support listing all solutions, but instead rather present a single one.
However, one can simply add the negation of the previous solution as another constraint and run the solver again.

Besides bechmarking the solvers, this notebook also demonstrates their API in general.

In [1]:
import time

def benchmark(name, function, param1, param2):
    """Benchmark a function with two parameters."""
    
    print('--'+ name + ' approach--')
    start_time = time.perf_counter()
    print('Number of models: ' + str(function(param1, param2)))
    end_time = time.perf_counter()
    print('Time: ' + str(round(end_time - start_time, 2)) + ' s')

## Z3

The [`Z3` library](https://github.com/Z3Prover/z3/wiki) offers SMT solving as well as optimization for various programming lnaguages.
The solver returns only one valid solution.
We try multiple ways to find/count all satisfying interpretations (models):
- Enumeration-based: Enumerate all possible assignments and check if they satisfy all assertions. This is possible in `Z3` with
  - conditional checking, which temporarily adds the literals corresponding to the assignment as further constraints and checks the resulting model.
  - substitution of the variables with their values, followed by simplification.
- [Solver-based](https://stackoverflow.com/questions/13395391/z3-finding-all-satisfying-models): Find the first model with the solver. Add the negation of its assignment as another constraint and re-run the solver. Thus, the solver will find a different solution. Repeat until unsatisfiable.

In [2]:
from z3 import *

def count_models_with_solver(solver, variables):
    solver.push() # as we will add further assertions to solver, checkpoint current state
    solutions = 0
    while solver.check() == sat:
        solutions = solutions + 1
        # Invert at least one variable to get a different solution:
        solver.add(Or([Not(x) if is_true(solver.model()[x]) else x for x in variables]))
    solver.pop() # restore solver to previous state
    return solutions

import itertools

# Fastest enumeration: conditional checking.
def count_models_by_enumeration(solver, variables):
    solutions = 0
    for assignment in itertools.product(*[(x, Not(x)) for x in variables]): # all combinations
        if solver.check(assignment) == sat: # conditional check (does not add assignment permanently)
            solutions = solutions + 1
    return solutions

# Creating the assignment as a separate step is slower.
def count_models_by_enumeration2(solver, variables):
    solutions = 0
    for assignment in itertools.product([False, True], repeat = len(variables)): # all combinations
        if solver.check([x if assign_true else Not(x) for x, assign_true in zip(variables, assignment)]) == sat:
            solutions = solutions + 1
    return solutions

# Using simplication instead of conditional checking is even slower.
def count_models_by_enumeration3(solver, variables):
    solutions = 0
    for assignment in itertools.product([BoolVal(False), BoolVal(True)], repeat = len(variables)): # all combinations
        satisfied = True
        for assertion in solver.assertions():
            if is_false(simplify(substitute(assertion, list(zip(variables, assignment))))):
                satisfied = False
                break
        if satisfied: solutions = solutions + 1
    return solutions

We try both approaches with a small propositional formula with 10 variables, using an AND constraint as well as an OR constraint.

In [3]:
from z3 import *

x = Bools(' '.join('x' + str(i) for i in range(10)))
solver = Solver()

print('## OR formula ##')
solver.add(Or(x))
benchmark('Solver-based', count_models_with_solver, solver, x)
benchmark('Enumeration-based (conditional check, direct assignment)', count_models_by_enumeration, solver, x)
benchmark('Enumeration-based (conditional check, separate assignment)', count_models_by_enumeration2, solver, x)
benchmark('Enumeration-based (substitute + simplify)', count_models_by_enumeration3, solver, x)

print('\n## AND formula ##')
solver.reset()
solver.add(And(x))
benchmark('Solver-based', count_models_with_solver, solver, x)
benchmark('Enumeration-based (conditional check, direct assignment)', count_models_by_enumeration, solver, x)
benchmark('Enumeration-based (conditional check, separate assignment)', count_models_by_enumeration2, solver, x)
benchmark('Enumeration-based (substitute + simplify)', count_models_by_enumeration3, solver, x)

## OR formula ##
--Solver-based approach--
Number of models: 1023
Time: 1.41 s
--Enumeration-based (conditional check, direct assignment) approach--
Number of models: 1023
Time: 0.02 s
--Enumeration-based (conditional check, separate assignment) approach--
Number of models: 1023
Time: 0.21 s
--Enumeration-based (substitute + simplify) approach--
Number of models: 1023
Time: 0.38 s

## AND formula ##
--Solver-based approach--
Number of models: 1
Time: 0.0 s
--Enumeration-based (conditional check, direct assignment) approach--
Number of models: 1
Time: 0.01 s
--Enumeration-based (conditional check, separate assignment) approach--
Number of models: 1
Time: 0.21 s
--Enumeration-based (substitute + simplify) approach--
Number of models: 1
Time: 0.38 s


The enumeration-based approach has to evaluate the same number of values for AND and OR formulas, i.e., all value combinations.
The conditional-checking approach still seems to benefit from problems with fewer solutions, though not as strong as the solver-based approach.
Overall, there is no clear winner: Depending on the number of solutions, solving or enumerating might be better.

## Google OR Tools

Google provides a framework for combinatorial white-box optimization problems, including constraint [solving](https://developers.google.com/optimization/cp/cp_solver) and [optimization](https://developers.google.com/optimization/cp/integer_opt_cp).
Besides the [Python API](https://developers.google.com/optimization/reference/python/sat/python/cp_model), C++, Java and C# are supported.
Creating an enumeration-based solution is more difficult than with `Z3`, as we cannot simply copy models or make conditional evaluations (temporary assignments).
Thus, we refrain from implementing such a solution.
Howeveer, as a nice alternative, iterating over all valid solutions is supported natively.

In [4]:
from ortools.sat.python import cp_model

def count_models_with_solver(model, variables):
    # TO DO: make a copy of model (not supported natively)
    solver = cp_model.CpSolver()
    solutions = 0
    while solver.Solve(model) == cp_model.FEASIBLE:
        solutions = solutions + 1
        # Invert at least one variable to get a different solution:
        model.AddBoolOr([x.Not() if solver.Value(x) == 1 else x for x in variables])
    return solutions

class Solution_Counter(cp_model.CpSolverSolutionCallback):

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1

    def solution_count(self):
        return self.__solution_count

def count_models_natively(model, counter_callback):
    solver = cp_model.CpSolver()
    solver.SearchForAllSolutions(model=model, callback=counter_callback)
    return counter_callback.solution_count()

In [5]:
from ortools.sat.python import cp_model

print('## OR formula ##')
model = cp_model.CpModel()
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolOr(x)
benchmark('Solver-based', count_models_with_solver, model, x)
model = cp_model.CpModel() # solver-based approach changes model, therefore re-creation
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolOr(x)
benchmark('Native', count_models_natively, model, Solution_Counter())

print('\n## AND formula ##')
model = cp_model.CpModel()
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolAnd(x)
benchmark('Solver-based', count_models_with_solver, model, x)
model = cp_model.CpModel()
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolAnd(x)
benchmark('Native', count_models_natively, model, Solution_Counter())

## OR formula ##
--Solver-based approach--
Number of models: 0
Time: 0.01 s
--Native approach--
Number of models: 1023
Time: 0.01 s

## AND formula ##
--Solver-based approach--
Number of models: 0
Time: 0.0 s
--Native approach--
Number of models: 1
Time: 0.0 s


The native approach beats the solver-based approach.
Compared to `Z3`, the native approach with `OR Tools` is about as fast as `Z3` (solver-based) for AND formulas and slightly faster than `Z3` (enumeration-based) for OR formulas.

## pySMT

[pySMT](https://pysmt.readthedocs.io/en/latest/index.html) is a wrapper for [various solvers](https://github.com/pysmt/pysmt#solvers-support) supporting the *SMT-Lib* format, including `Z3`.
The solvers have to be [installed separately](https://pysmt.readthedocs.io/en/latest/getting_started.html#installation).

In [6]:
from pysmt.shortcuts import *

def count_models_with_solver(solver, variables):
    solver.push() # as we will add further assertions to solver, checkpoint current state
    solutions = 0
    while solver.solve():
        solutions = solutions + 1
        # Invert at least one variable to get a different solution ("iff" == "<-->"):
        solver.add_assertion(Not(And([Iff(x, solver.get_value(x)) for x in variables])))
    solver.pop() # restore solver to previous state
    return solutions

import itertools

# Fastest enumeration by conditional checking.
def count_models_by_enumeration(solver, variables):
    solutions = 0
    for assignment in itertools.product(*[(x, Not(x)) for x in variables]): # all combinations
        if solver.solve(assignment): # conditional check (does not add assignment permanently)
            solutions = solutions + 1
    return solutions

# Slower enumeration by substitution and simplification.
def count_models_by_enumeration2(conditions, variables):
    solutions = 0
    for assignment in itertools.product([Bool(False), Bool(True)], repeat = len(variables)): # all combinations
        satisfied = True
        for condition in conditions:
            if substitute(condition, dict(zip(variables, assignment))).simplify().is_false():
                satisfied = False
                break
        if satisfied: solutions = solutions + 1
    return solutions

Here, we use the mSAT solver.

In [7]:
from pysmt.shortcuts import *

x = [Symbol('x' + str(i)) for i in range(10)]
solver = Solver(name='msat') # could also use 'z3'

print('## OR formula ##')
solver.add_assertion(Or(x))
benchmark('Solver-based', count_models_with_solver, solver, x)
benchmark('Enumeration-based (conditional check)', count_models_by_enumeration, solver, x)
benchmark('Enumeration-based (substitute + simplify)', count_models_by_enumeration2, [Or(x)], x)

print('\n## AND formula ##')
solver.reset_assertions()
solver.add_assertion(And(x))
benchmark('Solver-based', count_models_with_solver, solver, x)
benchmark('Enumeration-based (conditional check)', count_models_by_enumeration, solver, x)
benchmark('Enumeration-based (substitute + simplify)', count_models_by_enumeration2, [And(x)], x)

## OR formula ##
--Solver-based approach--
Number of models: 1023
Time: 0.58 s
--Enumeration-based (conditional check) approach--
Number of models: 1023
Time: 0.11 s
--Enumeration-based (substitute + simplify) approach--
Number of models: 1023
Time: 0.17 s

## AND formula ##
--Solver-based approach--
Number of models: 1
Time: 0.0 s
--Enumeration-based (conditional check) approach--
Number of models: 1
Time: 0.07 s
--Enumeration-based (substitute + simplify) approach--
Number of models: 1
Time: 0.18 s


We don't get a performance advantage compared to `Z3`.

## PicoSAT

A SAT solver implemented in C with a very simple [Python interface](https://github.com/ContinuumIO/pycosat).
It offers solving and iterative solving for pure SAT formulas in CNF.
The formula has to be provided as a list of clauses.
Each clause is a list of non-zero integers indicating the involved variables.
Negative numbers represent negated variables.

In [8]:
from pycosat import itersolve

def count_models_natively(formula, dummyParam):
    iterator = pycosat.itersolve(formula)
    return sum(1 for _ in iterator) # might be more efficient than turning into list

In [9]:
import pycosat

print('## OR formula ##')
orFormula = [[i for i in range(1, 11)]]
benchmark('Native', count_models_natively, orFormula, None)

print('\n## AND formula ##')
andFormula = [[i] for i in range(1, 11)]
benchmark('Native', count_models_natively, andFormula, None)

## OR formula ##
--Native approach--
Number of models: 1023
Time: 0.01 s

## AND formula ##
--Native approach--
Number of models: 1
Time: 0.0 s


A very fast solver, though the input requirements are strongly limiting.
The solver can also solve bigger OR clauses, e.g. with 20 variables, instantly, but iterating over the solutions then becomes the bottleneck.

## python-constraint

[python-constrant](https://labix.org/python-constraint) is directly targeted at constraint solving.
However, the pre-defined constraint types do not include logical constraints, which is no problem for simple scenarios (there are good arithmetic alternatives), but might become tricky when combining constraints.
It is possible to define your own constraints.

In [10]:
from constraint import *

# Constraints have to be classes or functions
class ExcludeSolutionConstraint(Constraint):
    
    def __init__(self, assignments):
        self._solution = assignments
        
    def __call__(self, variables, domains, assignments, forwardcheck=False):
        return assignments != self._solution # at least one difference

def count_models_with_solver(problem, variables):
    # TO DO: make a copy of problem (not supported natively)
    solutions = 0
    solution = problem.getSolution()
    while not solution is None:
        solutions = solutions + 1
        problem.addConstraint(ExcludeSolutionConstraint(solution))
        solution = problem.getSolution()
    return solutions

def count_models_natively(problem, dummyParam):
    return len(problem.getSolutions())

In [11]:
from constraint import *

x = ['x' + str(i) for i in range(10)]

print('## OR formula ##')
problem = Problem()
problem.addVariables(x, domain = [0,1])
problem.addConstraint(MinSumConstraint(1))
benchmark('Native', count_models_natively, problem, None)

print('\n## AND formula ##')
problem = Problem()
problem.addVariables(x, domain = [0,1])
problem.addConstraint(ExactSumConstraint(len(x)))
benchmark('Native', count_models_natively, problem, None)
benchmark('Solver-based', count_models_with_solver, problem, x)

## OR formula ##
--Native approach--
Number of models: 1023
Time: 0.02 s

## AND formula ##
--Native approach--
Number of models: 1
Time: 0.02 s
--Solver-based approach--
Number of models: 1
Time: 0.02 s


## Dummy Arithmetic Enumerator

To check how fast the enumeration loop is in general (without the solver overhead for conditional checking or simplification), we define some methods which each check one particular logical formula in an arithmetic way.
This is based on the observation that `AND` could be replaced with `min() == 1`, `OR` with `max() == 1` and `NOT(x)` with `1 - x`.

In [12]:
import itertools
import numpy as np

def count_models_by_enumeration_or(numVariables, dummyParam):
    solutions = 0
    for assignment in itertools.product([False, True], repeat=numVariables): # all combinations
        if max(assignment) == 1:
            solutions = solutions + 1
    return solutions

# Slower solution using vector operations: create assignment matrix
# (rows are assignments, columns are variables), evaluate row-wise
# and aggregate result
def count_models_by_enumeration_or2(numVariables, dummyParam):
    return np.array(list(itertools.product([False, True], repeat=numVariables))).max(axis=1).sum()

# Slower solution with manual int-to-binary conversion
# (using numpy array instead of list is even slower)
def count_models_by_enumeration_or3(numVariables, dummyParam):
    solutions = 0
    for i in range(2 ** numVariables):
        remaining_value = i
        for varIdx in range(numVariables):
            if remaining_value % 2 == 1:
                solutions = solutions + 1
                break # early ababdoning: if one variable true, then OR is true
            remaining_value = remaining_value // 2
    return solutions

def count_models_by_enumeration_and(numVariables, dummyParam):
    solutions = 0
    for assignment in itertools.product([False, True], repeat=numVariables): # all combinations
        if min(assignment) == 1:
            solutions = solutions + 1
    return solutions

In [13]:
print('## OR formula ##')
benchmark('Enumeration-based (10 variables)', count_models_by_enumeration_or, 10, None)
benchmark('Enumeration-based (20 variables)', count_models_by_enumeration_or, 20, None)

print('## AND formula ##')
benchmark('Enumeration-based (10 variables)', count_models_by_enumeration_and, 10, None)
benchmark('Enumeration-based (20 variables)', count_models_by_enumeration_and, 20, None)

## OR formula ##
--Enumeration-based (10 variables) approach--
Number of models: 1023
Time: 0.0 s
--Enumeration-based (20 variables) approach--
Number of models: 1048575
Time: 0.62 s
## AND formula ##
--Enumeration-based (10 variables) approach--
Number of models: 1
Time: 0.0 s
--Enumeration-based (20 variables) approach--
Number of models: 1
Time: 0.53 s


This iterative arithmetic approach is comparatively fast, but for each 10 new variables, the processing time will still increase $2^{10} = 1024$ times.
We can't prevent exponential growth ...
An alternative approach which uses vectorized evaluation of assignments still has to create huge assignment matrices.
This takes time and also consumes a lot of memory.
Plus, it only moves the starting point, the growth still happens ...

## Flexible Enumerator

The dummy enumerator above does not allow to build own logical expressions, it is tailored to hard-coded formulas.
Actually, is is not hard to create a configurable enumerator, which we do now.
First, we define several classes which allow to formulate nested constraints.
Next, we adapt our enumeration method as already used above.

In [14]:
# "interface" / super-class
class Expression:
    
    def is_true(self):
        pass # method not implemented here, but in each sub-class
    
class Variable(Expression):
    
    def __init__(self):
        self.value = False
    
    def is_true(self):
        return self.value

class Not(Expression):
    
    def __init__ (self, expression):
        self.__expression = expression
        
    def is_true(self):
        return not self.__expression.is_true()
    
class And(Expression):
    
    def __init__(self, expressions):
        self.__expressions = expressions
        
    def is_true(self):
        for expression in self.__expressions:
            if not expression.is_true():
                return False
        return True
    
class Or(Expression):
    
    def __init__(self, expressions):
        self.__expressions = expressions
        
    def is_true(self):
        for expression in self.__expressions:
            if expression.is_true():
                return True
        return False
    
import itertools

class Problem:
    
    def __init__(self, variables):
        self.__variables = variables
        self.__constraints = [] # several constraints allowed, will be combined by AND
    
    # Add an Expression as constraint
    def add_constraint(self, constraint):
        self.__constraints.append(constraint)
    
    def count_models_by_enumeration(self):
        solutions = 0
        for assignment in itertools.product([False, True], repeat=len(self.__variables)):
            # Assign
            for i in range(len(assignment)):
                self.__variables[i].value = assignment[i]
            # Check SAT
            satisfied = True
            for constraint in self.__constraints:
                if not constraint.is_true():
                    satisfied = False
                    break
            solutions = solutions + satisfied
        return solutions
    
def count_model_dispatch(problem, dummyParam):
    return problem.count_models_by_enumeration()

In [15]:
print('## OR formula ##')
x = [Variable() for i in range(10)]
problem = Problem(variables=x)
problem.add_constraint(Or(x))
benchmark('Enumeration-based (10 variables)', count_model_dispatch, problem, None)
x = [Variable() for i in range(20)]
problem = Problem(variables=x)
problem.add_constraint(Or(x))
benchmark('Enumeration-based (20 variables)', count_model_dispatch, problem, None)

print('## AND formula ##')
x = [Variable() for i in range(10)]
problem = Problem(variables=x)
problem.add_constraint(And(x))
benchmark('Enumeration-based (10 variables)', count_model_dispatch, problem, None)
x = [Variable() for i in range(20)]
problem = Problem(variables=x)
problem.add_constraint(And(x))
benchmark('Enumeration-based (20 variables)', count_model_dispatch, problem, None)

## OR formula ##
--Enumeration-based (10 variables) approach--
Number of models: 1023
Time: 0.0 s
--Enumeration-based (20 variables) approach--
Number of models: 1048575
Time: 4.36 s
## AND formula ##
--Enumeration-based (10 variables) approach--
Number of models: 1
Time: 0.0 s
--Enumeration-based (20 variables) approach--
Number of models: 1
Time: 4.58 s


We see some overhead compared to the arithmetic enumerator, but are still faster than any complete enumeration based on a solver.
We cannot beat iterative solving for formulas which only have a few models.